## Natural Language Processing for Text Classification with NLTK and Scikit-learn

In this project, we will expand on this foundation and explore different ways to improve our text classification results. We will cover and use:

* Regular Expressions
* Feature Engineering
* Multiple scikit-learn Classifiers
* Ensemble Methods


In [1]:
import sys
import nltk
import sklearn
import pandas as pd
import numpy as np

print('Python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pd.__version__))
print('Numpy: {}'.format(np.__version__))

Python: 3.7.3 (default, Apr 24 2019, 15:29:51) [MSC v.1915 64 bit (AMD64)]
NLTK: 3.4.4
Scikit-learn: 0.21.2
Pandas: 0.24.2
Numpy: 1.16.4


### 2. Load the Dataset

Now that we have ensured that our libraries are installed correctly, let's load the data set as a Pandas DataFrame. Furthermore, let's extract some useful information such as the column information and class distributions. 

The data set we will be using comes from the UCI Machine Learning Repository.  It contains over 5000 SMS labeled messages that have been collected for mobile phone spam research. It can be downloaded from the following URL:

https://archive.ics.uci.edu/ml/datasets/sms+spam+collection

In [2]:
# load the dataset of SMS messages
df = pd.read_csv('spam.csv', header=None,  encoding= "ISO-8859-1")
df.head()

,0,1,2,3,4
0,v1,v2,NaN,NaN,NaN
1,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
2,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
3,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
4,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN


In [3]:
# Remove unnecessary columns and rows
df = df.iloc[:, :2]
df = df.iloc[1:, :]
df.head()

,0,1
1,ham,"Go until jurong point, crazy.. Available only ..."
2,ham,Ok lar... Joking wif u oni...
3,spam,Free entry in 2 a wkly comp to win FA Cup fina...
4,ham,U dun say so early hor... U c already then say...
5,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
# check class distribution 
classes = df[0]
print(classes.value_counts())

ham     4825
spam     747
Name: 0, dtype: int64


### 2. Preprocess the Data

Preprocessing the data is an essential step in natural language process. In the following cells, we will convert our class labels to binary values using the LabelEncoder from sklearn, replace email addresses, URLs, phone numbers, and other symbols by using regular expressions, remove stop words, and extract word stems.  

In [5]:
#import Label Encoder from scikit learn 
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y = encoder.fit_transform(classes)
Y[:10]

array([0, 0, 1, 0, 0, 1, 0, 0, 1, 1])

In [6]:
# store the sms data
text_msg = df[1]
print(text_msg[:10])    

1     Go until jurong point, crazy.. Available only ...
2                         Ok lar... Joking wif u oni...
3     Free entry in 2 a wkly comp to win FA Cup fina...
4     U dun say so early hor... U c already then say...
5     Nah I don't think he goes to usf, he lives aro...
6     FreeMsg Hey there darling it's been 3 week's n...
7     Even my brother is not like to speak with me. ...
8     As per your request 'Melle Melle (Oru Minnamin...
9     WINNER!! As a valued network customer you have...
10    Had your mobile 11 months or more? U R entitle...
Name: 1, dtype: object


#### 2.1 Regular Expressions

Some common regular expression metacharacters - copied from wikipedia

**^**     Matches the starting position within the string. In line-based tools, it matches the starting position of any line.

**.**     Matches any single character (many applications exclude newlines, and exactly which characters are considered newlines is flavor-, character-encoding-, and platform-specific, but it is safe to assume that the line feed character is included). Within POSIX bracket expressions, the dot character matches a literal dot. For example, a.c matches "abc", etc., but [a.c] matches only "a", ".", or "c".

**[ ]**    A bracket expression. Matches a single character that is contained within the brackets. For example, [abc] matches "a", "b", or "c". [a-z] specifies a range which matches any lowercase letter from "a" to "z". These forms can be mixed: [abcx-z] matches "a", "b", "c", "x", "y", or "z", as does [a-cx-z].
The - character is treated as a literal character if it is the last or the first (after the ^, if present) character within the brackets: [abc-], [-abc]. Note that backslash escapes are not allowed. The ] character can be included in a bracket expression if it is the first (after the ^) character: []abc].

**[^ ]**   Matches a single character that is not contained within the brackets. For example, [^abc] matches any character other than "a", "b", or "c". [^a-z] matches any single character that is not a lowercase letter from "a" to "z". Likewise, literal characters and ranges can be mixed.

**$**      Matches the ending position of the string or the position just before a string-ending newline. In line-based tools, it matches the ending position of any line.

**( )**    Defines a marked subexpression. The string matched within the parentheses can be recalled later (see the next entry, \n). A marked subexpression is also called a block or capturing group. BRE mode requires \( \).

**\n**     Matches what the nth marked subexpression matched, where n is a digit from 1 to 9. This construct is vaguely defined in the POSIX.2 standard. Some tools allow referencing more than nine capturing groups.

**\***     Matches the preceding element zero or more times. For example, ab*c matches "ac", "abc", "abbbc", etc. [xyz]* matches "", "x", "y", "z", "zx", "zyx", "xyzzy", and so on. (ab)* matches "", "ab", "abab", "ababab", and so on.

**{m,n}**  Matches the preceding element at least m and not more than n times. For example, a{3,5} matches only "aaa", "aaaa", and "aaaaa". This is not found in a few older instances of regexes. BRE mode requires \{m,n\}.

In [7]:
# use regular expressions to replace email addresses, URLs, Phone numbers, other numbers

# Replace email addreses with 'email'
processed = text_msg.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$', 'emailaddress')

# Replace URLs with 'webaddress'
processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$','webaddress')

# Replace money symbols with 'moneysymb' (£ can by typed with ALT key + 156)
processed = processed.str.replace(r'£|\$', 'moneysymb')

# Replace 10 digit phone numbers (formats include paranthesis, spaces, no spaces, dashes) with 'phonenumber'
processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$','phonenumbr')
    
# Replace numbers with 'numbr'
processed = processed.str.replace(r'\d+(\.\d+)?', 'numbr')   



In [8]:
# Remove punctuation
processed = processed.str.replace(r'[^\w\d\s]', ' ')

# Replace whitespace between terms with a single space
processed = processed.str.replace(r'\s+', ' ')

# Remove leading and trailing whitespace
processed = processed.str.replace(r'^\s+|\s+?$', '')


In [9]:
# change words to lower case - Hello, HELLO, hello are all the same word
processed = processed.str.lower()
print(processed)

1       go until jurong point crazy available only in ...
2                                 ok lar joking wif u oni
3       free entry in numbr a wkly comp to win fa cup ...
4             u dun say so early hor u c already then say
5       nah i don t think he goes to usf he lives arou...
6       freemsg hey there darling it s been numbr week...
7       even my brother is not like to speak with me t...
8       as per your request melle melle oru minnaminun...
9       winner as a valued network customer you have b...
10      had your mobile numbr months or more u r entit...
11      i m gonna be home soon and i don t want to tal...
12      six chances to win cash from numbr to numbr nu...
13      urgent you have won a numbr week free membersh...
14      i ve been searching for the right words to tha...
15                      i have a date on sunday with will
16      xxxmobilemovieclub to use your credit click th...
17                                 oh k i m watching here
18      eh u r

In [10]:
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

# remove stop words from text messages

stop_words = set(stopwords.words('english'))

processed = processed.apply(lambda x : ' '.join(term for term in x.split() if term not in stop_words))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
# Remove word stems using a Porter stemmer

PS = nltk.PorterStemmer()

processed = processed.apply(lambda x: ' '.join(PS.stem(term) for term in x.split()))
print(processed[1])

go jurong point crazi avail bugi n great world la e buffet cine got amor wat


### 3. Generating Features

Feature engineering is the process of using domain knowledge of the data to create features for machine learning algorithms. In this project, the words in each text message will be our features.  For this purpose, it will be necessary to tokenize each word.  We will use the 1500 most common words as features.

In [12]:
from nltk.tokenize import word_tokenize

# create bags of words
all_words = []
for message in processed:
        words = word_tokenize(message)
        for w in words:
            all_words.append(w)

all_words = nltk.FreqDist(all_words)


In [13]:
# print the total number of words and the 15 most common words

print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(15)))

Number of words: 6538
Most common words: [('numbr', 2628), ('u', 1192), ('call', 672), ('go', 453), ('get', 451), ('ur', 385), ('gt', 318), ('lt', 316), ('come', 301), ('ok', 292), ('åmoneysymbnumbr', 288), ('free', 284), ('know', 274), ('day', 273), ('love', 260)]


In [14]:
# use the 1500 most common words as features

word_features = list(all_words.keys())[:1500]
print(word_features)

['go', 'jurong', 'point', 'crazi', 'avail', 'bugi', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'got', 'amor', 'wat', 'ok', 'lar', 'joke', 'wif', 'u', 'oni', 'free', 'entri', 'numbr', 'wkli', 'comp', 'win', 'fa', 'cup', 'final', 'tkt', 'numbrst', 'may', 'text', 'receiv', 'question', 'std', 'txt', 'rate', 'c', 'appli', 'numbrovernumbr', 'dun', 'say', 'earli', 'hor', 'alreadi', 'nah', 'think', 'goe', 'usf', 'live', 'around', 'though', 'freemsg', 'hey', 'darl', 'week', 'word', 'back', 'like', 'fun', 'still', 'tb', 'xxx', 'chg', 'send', 'åmoneysymbnumbr', 'rcv', 'even', 'brother', 'speak', 'treat', 'aid', 'patent', 'per', 'request', 'mell', 'oru', 'minnaminungint', 'nurungu', 'vettam', 'set', 'callertun', 'caller', 'press', 'copi', 'friend', 'winner', 'valu', 'network', 'custom', 'select', 'receivea', 'prize', 'reward', 'claim', 'call', 'code', 'klnumbr', 'valid', 'hour', 'mobil', 'month', 'r', 'entitl', 'updat', 'latest', 'colour', 'camera', 'co', 'gon', 'na', 'home', 'soon', 'wan

In [15]:
# The find_features function will determine which of the 1500 words features are contained in the review

def find_features(message):
    words = word_tokenize(message)
    feature = {}
    for word in word_features:
        feature[word] = (word in words)
    return feature

# Lets see an example
features = find_features(processed[1])
for key, value in features.items():
    if value == True:
        print(key)

go
jurong
point
crazi
avail
bugi
n
great
world
la
e
buffet
cine
got
amor
wat


In [16]:
# Now lets do it for all the messages
messages = list(zip(processed, Y))

# define a seed for reproducibility
seed = 1
np.random.seed = seed
np.random.shuffle(messages)

# call find_features function for each SMS message
featuresets = [(find_features(text), label) for (text, label) in messages]
print(featuresets[10])

({'go': False, 'jurong': False, 'point': False, 'crazi': False, 'avail': False, 'bugi': False, 'n': False, 'great': False, 'world': False, 'la': False, 'e': False, 'buffet': False, 'cine': False, 'got': False, 'amor': False, 'wat': False, 'ok': False, 'lar': False, 'joke': False, 'wif': False, 'u': True, 'oni': False, 'free': False, 'entri': False, 'numbr': True, 'wkli': False, 'comp': False, 'win': False, 'fa': False, 'cup': False, 'final': False, 'tkt': False, 'numbrst': True, 'may': False, 'text': True, 'receiv': False, 'question': False, 'std': False, 'txt': True, 'rate': False, 'c': False, 'appli': False, 'numbrovernumbr': False, 'dun': False, 'say': False, 'earli': False, 'hor': False, 'alreadi': False, 'nah': False, 'think': False, 'goe': False, 'usf': False, 'live': False, 'around': False, 'though': False, 'freemsg': False, 'hey': False, 'darl': False, 'week': False, 'word': False, 'back': False, 'like': True, 'fun': True, 'still': False, 'tb': False, 'xxx': False, 'chg': False

In [17]:
# we can split featuresets into train and test sets
from sklearn.model_selection import train_test_split

training, testing = train_test_split(featuresets, test_size = 0.25, random_state = seed)


In [18]:
# print lenth of training and testing set
print(len(training))
print(len(testing))

4179
1393


### 4. Scikit-Learn Classifiers with NLTK

Now that we have our dataset, we can start building algorithms! Let's start with a simple linear support vector classifier, then expand to other algorithms. We'll need to import each algorithm we plan on using from sklearn.  We also need to import some performance metrics, such as accuracy_score and classification_report.

In [19]:
# we can use sklearn algorithms in NLTK
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import SVC

model = SklearnClassifier(SVC(kernel = 'linear'))

# train the model on the training data
model.train(training)

# and test on the testing dataset !
accuracy = nltk.classify.accuracy(model, testing)*100
print("SVC Accuracy: {}".format(accuracy))

SVC Accuracy: 98.34888729361091


In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Define models to train
names = ["SVM","K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifier = [SVC(kernel = 'linear'), KNeighborsClassifier(), DecisionTreeClassifier(), RandomForestClassifier(), LogisticRegression(),  SGDClassifier(max_iter = 100), MultinomialNB(), SVC(kernel = 'linear')]
models = zip(names, classifier)
for name, model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing)*100
    print("{} Accuracy: {}".format(name, accuracy))

SVM Accuracy: 98.34888729361091
K Nearest Neighbors Accuracy: 95.11844938980617
Decision Tree Accuracy: 97.4156496769562


C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Random Forest Accuracy: 98.34888729361091


C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy: 98.56424982053123
SGD Classifier Accuracy: 98.06173725771716
Naive Bayes Accuracy: 98.56424982053123
SVM Linear Accuracy: 98.34888729361091


In [21]:
# Ensemble method - Voting classifier
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))

nltk_ensemble = SklearnClassifier(VotingClassifier(estimators = models, voting = 'hard', n_jobs = -1 ))
nltk_ensemble.train(training)
accuracy = nltk.classify.accuracy(nltk_ensemble, testing)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

Voting Classifier: Accuracy: 98.42067480258436


In [22]:
# make class label prediction for testing set
txt_features, labels = zip(*testing)

prediction = nltk_ensemble.classify_many(txt_features)

In [23]:
# print a confusion matrix and a classification report
print(classification_report(labels, prediction))

pd.DataFrame(
    confusion_matrix(labels, prediction),
    index = [['actual', 'actual'], ['ham', 'spam']],
    columns = [['predicted', 'predicted'], ['ham', 'spam']])

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1215
           1       0.97      0.90      0.94       178

    accuracy                           0.98      1393
   macro avg       0.98      0.95      0.96      1393
weighted avg       0.98      0.98      0.98      1393



predicted     
                  ham spam
actual ham       1210    5
       spam        17  161

#### Thank you!